In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


#### For people who may not have access to their own medical data 

In [4]:
#age, sex, thalach using formulas to cal
# To estimate your maximum age-related heart rate, subtract your age from 220. For example, for a 50-year-old person, the estimated maximum age-related heart rate would be calculated as 220 – 50 years = 170 beats per minute (bpm).

In [5]:
#54, m

In [6]:
df_2 = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_2[(df_2['sex']==1)& (df_2['age'] ==54)].head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [7]:
df.chol.describe()
# 1 = male, 0 = female
df1 = df.groupby(by = ['age', 'sex'])['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'].mean()
df_2[(df_2['sex'] == 1) & (df_2['age'] == 54)].reset_index()

/tmp/ipykernel_2034/883282141.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = df.groupby(by = ['age', 'sex'])['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'].mean()


,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [8]:
df1

cp    trestbps        chol       fbs   restecg     exang  \
age sex                                                                   
29  1    1.000000  130.000000  204.000000  0.000000  0.000000  0.000000   
34  0    1.000000  118.000000  210.000000  0.000000  1.000000  0.000000   
    1    3.000000  118.000000  182.000000  0.000000  0.000000  0.000000   
35  0    0.000000  138.000000  183.000000  0.000000  1.000000  0.000000   
    1    0.363636  122.363636  218.727273  0.000000  0.727273  0.636364   
...           ...         ...         ...       ...       ...       ...   
70  1    0.642857  146.285714  251.857143  0.000000  0.500000  0.500000   
71  0    1.000000  124.363636  232.909091  0.363636  0.636364  0.000000   
74  0    1.000000  120.000000  269.000000  0.000000  0.000000  1.000000   
76  0    2.000000  140.000000  197.000000  0.000000  2.000000  0.000000   
77  1    0.000000  125.000000  304.000000  0.000000  0.000000  1.000000   

          oldpeak     slope        ca      thal    target  
age sex                                                    
29  1    0.000000  2.000000  0.000000  2.000000  1.000000  
34  0    0.700000  2.000000  0.000000  2.000000  1.000000  
    1    0.000000  2.000000  0.000000  2.000000  1.000000  
35  0    1.400000  2.000000  0.000000  2.000000  1.000000  
    1    0.581818  1.636364  0.000000  2.636364  0.363636  
...           ...       ...       ...       ...       ...  
70  1    2.050000  0.928571  1.071429  2.500000  0.214286  
71  0    0.690909  1.636364  0.909091  2.000000  1.000000  
74  0    0.200000  2.000000  1.000000  2.000000  1.000000  
76  0    1.100000  1.000000  0.000000  2.000000  1.000000  
77  1    0.000000  2.000000  3.000000  2.000000  0.000000  

[73 rows x 11 columns]

In [9]:
#function that takes in age and sex and returns a row of the mean data from that groupby df

In [10]:
# def mean_data:
#    user_age = input("Age?")
#    user_sex = input("Sex?")

In [11]:
df_mean_data = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_mean_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,1,1.000000,130.000000,204.000000,0.000000,0.000000,202.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
1,34,0,1.000000,118.000000,210.000000,0.000000,1.000000,192.000000,0.000000,0.700000,2.000000,0.000000,2.000000,1.000000
2,34,1,3.000000,118.000000,182.000000,0.000000,0.000000,174.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
3,35,0,0.000000,138.000000,183.000000,0.000000,1.000000,182.000000,0.000000,1.400000,2.000000,0.000000,2.000000,1.000000
4,35,1,0.363636,122.363636,218.727273,0.000000,0.727273,153.090909,0.636364,0.581818,1.636364,0.000000,2.636364,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,70,1,0.642857,146.285714,251.857143,0.000000,0.500000,121.500000,0.500000,2.050000,0.928571,1.071429,2.500000,0.214286
69,71,0,1.000000,124.363636,232.909091,0.363636,0.636364,136.909091,0.000000,0.690909,1.636364,0.909091,2.000000,1.000000
70,74,0,1.000000,120.000000,269.000000,0.000000,0.000000,121.000000,1.000000,0.200000,2.000000,1.000000,2.000000,1.000000
71,76,0,2.000000,140.000000,197.000000,0.000000,2.000000,116.000000,0.000000,1.100000,1.000000,0.000000,2.000000,1.000000


In [12]:
df1=df_mean_data[(df_mean_data['sex'] == 1) & (df_mean_data['age'] == 54)]
df1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [13]:
#print("Please Enter Sex")
#ans_sex = input("Sex")

# male


Please Enter Sex


Sex 

KeyboardInterrupt: Interrupted by user

In [0]:
def specify():
    
    print("Please Enter Sex")
    ans_sex = input("Sex")
    if ans_sex.lower() in ['male']:
        s = 1
        
    elif ans_sex.lower() in ['female']:
        s = 0
    
    print("Please Enter Age")
    ans_age = input("Age")
    a = ans_age
    
    
    
    
    new_df = df_mean_data[(df_mean_data['sex'] == s) & (df_mean_data['age'] == a)]
    
    new_df.head()

In [0]:
specify()

####



In [0]:
df_2[(df_2['sex']==1)& (df_2['age'] ==54)].head()

In [0]:
age_sex(age,sex)

In [0]:
df_mean_data.head()

In [0]:
def specify1():
    
    print("Please Enter Sex")
    ans_sex = input("Sex")
    if ans_sex.lower() in ['male']:
        s = 1
        
    elif ans_sex.lower() in ['female']:
        s = 0
    
    print("Please Enter Age")
    ans_age = input("Age")
    a = ans_age
    
    
    
    
    new_df = df_mean_data[(df_mean_data['sex'] == 1) & (df_mean_data['age'] == 54)].head()
    
    new_df.head()

In [0]:
specify1()
    #i think specify2 works

#####



In [24]:
def specify2():
    
    print("Please Enter Sex")
    ans_sex = input("Sex")
    if ans_sex.lower() in ['male']:
        s = 1
        
    elif ans_sex.lower() in ['female']:
        s = 0
    else:
        exit()
    
    print("Please Enter Age")
    ans_age = input("Age")
    
    if ans_age < 29 or ans_age > 77:
        print("No data for that age")
        exit() 
        
    a = int(ans_age)
    
    
    
    
    new_df = df_mean_data[(df_mean_data['sex'] == s) & (df_mean_data['age'] == a)]
    return new_df

In [16]:
mean_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [17]:
specify2()

Please Enter Sex


Sex female

Please Enter Age


Age 46

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
21,46,0,1.0,128.333333,208.0,0.0,0.333333,161.333333,0.666667,0.466667,1.0,0.0,2.0,1.0


In [19]:
df.age.describe()

count    1025.000000
mean       54.434146
std         9.072290
min        29.000000
25%        48.000000
50%        56.000000
75%        61.000000
max        77.000000
Name: age, dtype: float64

minimum age = 29 <br>
maximum age = 77 